In [17]:
import torch
print(torch.backends.mps.is_available())

True


In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# Set device to MPS if available, else use CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load your dataset
df = pd.read_csv('dataset.csv')
df.dropna(subset=['Content_1', 'Genre'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Encode labels
label_encoder = LabelEncoder()
df['Genre_encoded'] = label_encoder.fit_transform(df['Genre'])

# Split the dataset
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

class MovieGenreDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = dataframe['Content_1'].tolist()
        self.labels = dataframe['Genre_encoded'].tolist()
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        # Move tensors to the appropriate device
        return {
            'input_ids': encoding['input_ids'].to(device).squeeze(),
            'attention_mask': encoding['attention_mask'].to(device).squeeze(),
            'labels': torch.tensor(labels, dtype=torch.long).to(device)
        }

# Prepare data loaders
train_dataset = MovieGenreDataset(train_df, tokenizer)
val_dataset = MovieGenreDataset(val_df, tokenizer)
test_dataset = MovieGenreDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased', num_labels=len(label_encoder.classes_)).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    no_cuda=True  # Ensure CUDA is not used even if available
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(test_dataset)
print(results)

# Save the model and tokenizer
model.save_pretrained('./parsbert_movie_genre_classifier')
tokenizer.save_pretrained('./parsbert_movie_genre_classifier')


/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/transformers/training_args.py:1509: FutureWarning: using `no_cuda` is deprecated and will be removed in 

{'loss': 2.7766, 'grad_norm': 13.639311790466309, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


                                                
  6%|▌         | 20/324 [01:54<28:41,  5.66s/it]

{'loss': 2.8095, 'grad_norm': 11.837835311889648, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}


                                                
  9%|▉         | 30/324 [02:49<27:03,  5.52s/it]

...

100%|██████████| 324/324 [30:20<00:00,  4.65s/it]















                                                 
                                    
100%|██████████| 324/324 [30:35<00:00,  4.65s/it]
                                                 
100%|██████████| 324/324 [30:35<00:00,  5.67s/it]


{'eval_loss': 1.2991434335708618, 'eval_runtime': 13.2341, 'eval_samples_per_second': 8.161, 'eval_steps_per_second': 1.058, 'epoch': 3.0}
{'train_runtime': 1835.7278, 'train_samples_per_second': 1.405, 'train_steps_per_second': 0.176, 'train_loss': 1.6375918064588382, 'epoch': 3.0}


100%|██████████| 14/14 [00:11<00:00,  1.22it/s]

{'eval_loss': 1.1904798746109009, 'eval_runtime': 12.4938, 'eval_samples_per_second': 8.644, 'eval_steps_per_second': 1.121, 'epoch': 3.0}


('./parsbert_movie_genre_classifier/tokenizer_config.json',
 './parsbert_movie_genre_classifier/special_tokens_map.json',
 './parsbert_movie_genre_classifier/vocab.txt',
 './parsbert_movie_genre_classifier/added_tokens.json',
 './parsbert_movie_genre_classifier/tokenizer.json')

In [26]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

predictions = trainer.predict(test_dataset)
y_true = test_df['Genre_encoded'].to_numpy()
y_pred = np.argmax(predictions.predictions, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true, y_pred)

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'F1 Score (Macro): {f1_macro}')
print(f'F1 Score (Micro): {f1_micro}')
print(f'Precision (Macro): {precision}')
print(f'Recall (Macro): {recall}')
print('Confusion Matrix:')
print(conf_matrix)

100%|██████████| 14/14 [00:13<00:00,  1.02it/s]

Accuracy: 0.6388888888888888
F1 Score (Macro): 0.212359900373599
F1 Score (Micro): 0.6388888888888888
Precision (Macro): 0.2773109243697479
Recall (Macro): 0.2091723093371347
Confusion Matrix:
[[ 2  0  1  0  5  0  0  0]
 [ 0  0  0  0  5  0  0  0]
 [ 0  0 12  0 11  0  0  0]
 [ 0  0  2  0  5  0  0  0]
 [ 0  0  6  0 55  0  0  0]
 [ 0  0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  1  0  0  0]]



/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
